In [ ]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import pandas as pd
import yaml

import matplotlib.pyplot as plt
import seaborn as sns

from torch_geometric.data import Batch

from torch_geometric.data import Data

from gnn4itk_cf.stages.edge_classifier import InteractionGNN

In [ ]:
from torch_geometric.loader import ShaDowKHopSampler

config = yaml.load(open('gnn_train.yaml'), Loader=yaml.FullLoader)
model = InteractionGNN(config)
model.setup('fit')
trainsetList = []

for data1 in model.trainset:
    my_data1 = Data(hit_id=data1["hit_id"],x=data1["x"], r=data1["r"], phi=data1["phi"], eta=data1["eta"], y=data1["y"], z=data1["z"], edge_index=data1["edge_index"], truth_map=data1["truth_map"],weights=data1["weights"])
    trainsetList.append(my_data1)  #my_data1
trainset = Batch.from_data_list(trainsetList)

depth = 2
num_neighbors = 5
batch_size = 1

sampler = ShaDowKHopSampler(trainset,depth = depth,num_neighbors=num_neighbors,node_idx = None,replace=False,batch_size=batch_size, shuffle=True)

/users/PLS0150/ialadutska/data/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [particle_id] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/users/PLS0150/ialadutska/data/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [nhits] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/users/PLS0150/ialadutska/data/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [primary] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/users/PLS0150/ialadutska/data/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [pdgId] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/users/PLS0150/ialadutska/data/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [ghost] not found in data
  warnings.warn(f"O

Defining figures of merit


/users/PLS0150/ialadutska/data/CommonFramework/gnn4itk_cf/stages/edge_classifier/edge_classifier_stage.py:74: UserWarning: Failed to define figures of merit, due to logger unavailable
  warnings.warn("Failed to define figures of merit, due to logger unavailable")


In [12]:
total_num_nodes = 0
total_num_edges = 0
results = []
for step, sub_data in enumerate(sampler):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of nodes in the current batch: {sub_data.num_nodes} and edges {sub_data.num_edges}')
    print(sub_data)
    print()
    results.append(
        {
            'step': step,
            'Nodes': sub_data.num_nodes,
            'Edges':  sub_data.num_edges
        })
    total_num_nodes += sub_data.num_nodes
    total_num_edges += sub_data.num_edges
    if step == 4:
        break
    

resultsDF =pd.DataFrame(results)
plt.figure()
sns.histplot(resultsDF, x="Nodes", y="Edges",cbar=True)
print("Number of Neighbors: ", num_neighbors)
print("Batch size: ", batch_size, " Steps: ", step+1)
print(f'Iterated over {total_num_nodes} of {trainset.num_nodes} nodes!')
print(f'Iterated over {total_num_edges} of {trainset.num_edges} edges!')

In [22]:
resultsDF.head(10)

,step,Nodes,Edges
0,0,11311,22062
1,1,11198,21779
2,2,11414,22682
3,3,11452,22432
4,4,11211,22185
